In [1]:
import pandas as pd 
import tensorflow as tf
import numpy as np
pd.set_option("display.max_columns",40)

In [2]:
train = pd.read_csv("/dw-data/make-ipinyou-data/3358/train.log.txt",delimiter='\t',na_values=["null","NaN"],dtype="str")
train = train.drop(["bidid","urlid","ipinyouid","advertiser","logtype","url","bidprice","payprice","timestamp","keypage"],axis=1)
train = train.dropna()
train["slotprice"] = train["slotprice"].astype(int)
train["click"] = train["click"].astype(int)
train_labels=train["click"]
train.head()

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,usertag
0,0,4,00,windows_ie,222.220.35.*,308,320,2,eSMvBpa0jqmUagk4JKTI,3521017083,336,280,2,0,5,13606a7c541dcd9ca1948875a760bb31,"13800,10024"
1,0,4,00,windows_chrome,58.100.240.*,94,95,1,tK1NTu1YP5scFsf,mm_10075660_3500949_11453278,950,90,0,1,0,d5cecca9a6cbd7a0a48110f1306b26d1,"10059,13866,10063,10111"
2,0,4,00,android_opera,58.67.157.*,216,217,1,trqRTuS8jZL7FmMs,mm_40468498_3493773_11415894,300,250,0,5,0,d881a6c788e76c2c27ed1ef04f119544,"13866,10111"
4,0,4,00,windows_ie,118.182.244.*,344,350,1,DFpETuxoGQdcFNKbuKz,mm_10024662_3445902_11178345,336,280,2,1,0,13606a7c541dcd9ca1948875a760bb31,"10006,10110"
5,0,4,00,windows_ie,114.250.252.*,1,1,2,trqRTvKoMNmYgMn4JKTI,3443576454,300,250,1,0,5,d881a6c788e76c2c27ed1ef04f119544,"10006,10111"


In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1319249 entries, 0 to 1742102
Data columns (total 17 columns):
click             1319249 non-null int64
weekday           1319249 non-null object
hour              1319249 non-null object
useragent         1319249 non-null object
IP                1319249 non-null object
region            1319249 non-null object
city              1319249 non-null object
adexchange        1319249 non-null object
domain            1319249 non-null object
slotid            1319249 non-null object
slotwidth         1319249 non-null object
slotheight        1319249 non-null object
slotvisibility    1319249 non-null object
slotformat        1319249 non-null object
slotprice         1319249 non-null int64
creative          1319249 non-null object
usertag           1319249 non-null object
dtypes: int64(2), object(15)
memory usage: 181.2+ MB


In [40]:
len(train["usertag"].unique())

376724

In [ ]:
def input_fn(data_file, num_epochs, shuffle):
  """Input builder function."""
  df_data = pd.read_csv(
      tf.gfile.Open(data_file),
      names=CSV_COLUMNS,
      skipinitialspace=True,
      engine="python",
      skiprows=1)
  # remove NaN elements
  df_data = df_data.dropna(how="any", axis=0)
  labels = df_data["income_bracket"].apply(lambda x: ">50K" in x).astype(int)
  return tf.estimator.inputs.pandas_input_fn(
      x=df_data,
      y=labels,
      batch_size=100,
      num_epochs=num_epochs,
      shuffle=shuffle,
      num_threads=24)

In [3]:
weekday = tf.feature_column.categorical_column_with_vocabulary_list(
    "weekday",['4', '5', '6', '0', '1', '2', '3'])

hour = tf.feature_column.categorical_column_with_vocabulary_list(
"hour",['00', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '01','22', '23'])

useragent= tf.feature_column.categorical_column_with_vocabulary_list(
"useragent",['windows_ie', 'windows_chrome', 'android_opera', 'windows_opera',
       'android_safari', 'mac_chrome', 'mac_safari', 'windows_firefox',
       'mac_other', 'windows_theworld', 'windows_other', 'windows_maxthon',
       'android_other', 'linux_chrome', 'android_chrome', 'mac_firefox',
       'other_other', 'other_opera', 'windows_safari', 'android_firefox',
       'linux_safari', 'ios_other', 'ios_safari', 'linux_firefox',
       'mac_maxthon', 'other_safari', 'linux_other', 'windows_sogou',
       'other_chrome', 'android_maxthon', 'linux_opera', 'android_ie',
       'mac_opera', 'android_sogou', 'mac_sogou', 'other_ie',
       'other_firefox'])

IP = tf.feature_column.categorical_column_with_hash_bucket(
"IP",450000)

region = tf.feature_column.categorical_column_with_vocabulary_list(
"region",["0","1","2","3","15","27","40","55","65","79","80","94","106",
          "124","134","146","164","183","201","216","238","253","275","276",
          "298","308","325","333","344","359","368","374","393","394","395"])

city = tf.feature_column.categorical_column_with_vocabulary_list(
"city",['0','4','5','6','7','8','9','10','11','12','13','14','16','17',
        '18','19','20','21','22','23','24','25','26','28','29','30','31',
        '32','33','34','35','36','37','38','39','41','42','43','44','45',
        '46','47','48','49','50','51','52','53','54','56','57','58','59',
        '60','61','62','63','64','66','67','68','69','70','71','72','73',
        '74','75','76','77','78','81','82','83','84','85','86','87','88',
        '89','90','91','92','93','95','96','97','98','99','100','101','102',
        '103','104','105','107','108','109','110','111','112','113','114','115',
        '116','117','118','119','120','121','122','123','125','126','127','128',
        '129','130','131','132','133','135','136','137','138','139','140','141',
        '142','143','144','145','147','148','149','150','151','152','153','154',
        '155','156','157','158','159','160','161','162','163','165','166','167',
        '168','169','170','171','172','173','174','175','176','177','178','179',
        '180','181','182','184','185','186','187','188','189','190','191','192',
        '193','194','195','196','197','198','199','200','202','203','204','205',
        '206','207','208','209','210','211','212','213','214','215','217','218',
        '219','220','221','222','223','224','225','226','227','228','229','230',
        '231','232','233','234','235','236','237','239','240','241','242','243',
        '244','245','246','247','248','249','250','251','252','254','255','256',
        '257','258','259','260','261','262','263','264','265','266','267','268',
        '269','270','271','272','273','274','277','278','279','280','281','282',
        '283','284','285','286','287','288','289','290','291','292','293','294',
        '295','296','297','299','300','301','302','303','304','305','306','307',
        '309','310','311','312','313','314','315','316','317','318','319','320',
        '321','322','323','324','326','327','328','329','330','331','332','334',
        '335','336','337','338','339','340','341','342','343','345','346','347',
        '348','349','350','351','352','353','354','355','356','357','358','360',
        '361','362','363','364','365','366','367','369','370','371','372','373',
        '375','376','377','378','379','380','381','382','383','384','385','386',
        '387','388','389','390','391','392'])

adexchange = tf.feature_column.categorical_column_with_vocabulary_list(
"adexchange",['2', '1', '3'])

domain = tf.feature_column.categorical_column_with_hash_bucket(
"domain",11500)

slotid = tf.feature_column.categorical_column_with_hash_bucket(
"slotid",34500)

slotwidth = tf.feature_column.categorical_column_with_vocabulary_list(
"slotwidth",['336', '950', '300', '728', '160', '200', '250', '120', '1000',
       '360', '960'])

slotheight = tf.feature_column.categorical_column_with_vocabulary_list(
"slotheight",['280', '90', '250', '600', '200', '300'])

slotvisibility = tf.feature_column.categorical_column_with_vocabulary_list(
"slotvisibility",['2', '0', '1', '255'])

slotformat = tf.feature_column.categorical_column_with_vocabulary_list(
"slotformat",['0', '1', '5'])

slotprice = tf.feature_column.numeric_column("slotprice")
slotprice_buckets = tf.feature_column.bucketized_column(
slotprice,[0,1,10,50,100])

creative = tf.feature_column.categorical_column_with_hash_bucket(
"creative",50)

usertag = tf.feature_column.categorical_column_with_hash_bucket(
"usertag",380000)

base_columns = [weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,
               slotheight,slotvisibility,slotformat,slotprice_buckets,creative,usertag]

model_dir = "./linear_model"
m = tf.estimator.LinearClassifier(
feature_columns=base_columns,model_dir=model_dir,optimizer=tf.train.FtrlOptimizer(
      learning_rate=1e-3,
      l1_regularization_strength=1e-4))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './linear_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [13]:
m.

TypeError: 'RunConfig' object is not callable

In [4]:
# train
tf_train = tf.estimator.inputs.pandas_input_fn(
      x=train,
      y=train_labels,
      batch_size=1,
      num_epochs=1,
      shuffle=True,
      num_threads=1)
# set num_epochs to None to get infinite stream of data.
m.train(input_fn=tf_train)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./linear_model/model.ckpt.
INFO:tensorflow:loss = 0.693147, step = 1
INFO:tensorflow:global_step/sec: 262.357
INFO:tensorflow:loss = 0.616236, step = 101 (0.392 sec)
INFO:tensorflow:global_step/sec: 380.64
INFO:tensorflow:loss = 0.585442, step = 201 (0.261 sec)
INFO:tensorflow:global_step/sec: 367.947
INFO:tensorflow:loss = 0.625286, step = 301 (0.272 sec)
INFO:tensorflow:global_step/sec: 338.612
INFO:tensorflow:loss = 0.52851, step = 401 (0.295 sec)
INFO:tensorflow:global_step/sec: 335.338
INFO:tensorflow:loss = 0.520455, step = 501 (0.298 sec)
INFO:tensorflow:global_step/sec: 335.78
INFO:tensorflow:loss = 0.493583, step = 601 (0.298 sec)
INFO:tensorflow:global_step/sec: 337.2
INFO:tensorflow:loss = 0.493493, step = 701 (0.297 sec)
INFO:tensorflow:global_step/sec: 337.915
INFO:tensorflow:loss = 0.486053, step = 801 (0.296 sec)
INFO:tensorflow:global_step/sec: 339.606
INFO:tensorflow:loss = 0.4638

INFO:tensorflow:loss = 0.2016, step = 8301 (0.317 sec)
INFO:tensorflow:global_step/sec: 326.742
INFO:tensorflow:loss = 0.19567, step = 8401 (0.306 sec)
INFO:tensorflow:global_step/sec: 318.212
INFO:tensorflow:loss = 0.219475, step = 8501 (0.314 sec)
INFO:tensorflow:global_step/sec: 321.839
INFO:tensorflow:loss = 0.217133, step = 8601 (0.311 sec)
INFO:tensorflow:global_step/sec: 310.857
INFO:tensorflow:loss = 0.202545, step = 8701 (0.322 sec)
INFO:tensorflow:global_step/sec: 312.471
INFO:tensorflow:loss = 0.238329, step = 8801 (0.320 sec)
INFO:tensorflow:global_step/sec: 317.919
INFO:tensorflow:loss = 0.216666, step = 8901 (0.315 sec)
INFO:tensorflow:global_step/sec: 320.644
INFO:tensorflow:loss = 0.204893, step = 9001 (0.312 sec)
INFO:tensorflow:global_step/sec: 324.182
INFO:tensorflow:loss = 0.209485, step = 9101 (0.309 sec)
INFO:tensorflow:global_step/sec: 346.345
INFO:tensorflow:loss = 0.250373, step = 9201 (0.291 sec)
INFO:tensorflow:global_step/sec: 325.675
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 331.136
INFO:tensorflow:loss = 0.127447, step = 16701 (0.302 sec)
INFO:tensorflow:global_step/sec: 324.898
INFO:tensorflow:loss = 0.143512, step = 16801 (0.308 sec)
INFO:tensorflow:global_step/sec: 328.438
INFO:tensorflow:loss = 0.148196, step = 16901 (0.304 sec)
INFO:tensorflow:global_step/sec: 323.569
INFO:tensorflow:loss = 0.1315, step = 17001 (0.309 sec)
INFO:tensorflow:global_step/sec: 332.448
INFO:tensorflow:loss = 0.182119, step = 17101 (0.301 sec)
INFO:tensorflow:global_step/sec: 329.629
INFO:tensorflow:loss = 0.160888, step = 17201 (0.303 sec)
INFO:tensorflow:global_step/sec: 332.098
INFO:tensorflow:loss = 0.154507, step = 17301 (0.301 sec)
INFO:tensorflow:global_step/sec: 309.638
INFO:tensorflow:loss = 0.146843, step = 17401 (0.323 sec)
INFO:tensorflow:global_step/sec: 331.942
INFO:tensorflow:loss = 0.12893, step = 17501 (0.301 sec)
INFO:tensorflow:global_step/sec: 332.403
INFO:tensorflow:loss = 0.130043, step = 17601 (0.301 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 325.658
INFO:tensorflow:loss = 0.0992387, step = 25001 (0.307 sec)
INFO:tensorflow:global_step/sec: 323.45
INFO:tensorflow:loss = 0.0997117, step = 25101 (0.309 sec)
INFO:tensorflow:global_step/sec: 330.51
INFO:tensorflow:loss = 0.105908, step = 25201 (0.303 sec)
INFO:tensorflow:global_step/sec: 332.366
INFO:tensorflow:loss = 0.108997, step = 25301 (0.301 sec)
INFO:tensorflow:global_step/sec: 316.854
INFO:tensorflow:loss = 0.101134, step = 25401 (0.316 sec)
INFO:tensorflow:global_step/sec: 330.066
INFO:tensorflow:loss = 0.101277, step = 25501 (0.303 sec)
INFO:tensorflow:global_step/sec: 335.271
INFO:tensorflow:loss = 0.108279, step = 25601 (0.298 sec)
INFO:tensorflow:global_step/sec: 331.676
INFO:tensorflow:loss = 0.106952, step = 25701 (0.302 sec)
INFO:tensorflow:global_step/sec: 325.934
INFO:tensorflow:loss = 0.0982317, step = 25801 (0.307 sec)
INFO:tensorflow:global_step/sec: 327.228
INFO:tensorflow:loss = 0.0992621, step = 25901 (0.305 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 331.102
INFO:tensorflow:loss = 0.0758028, step = 33301 (0.302 sec)
INFO:tensorflow:global_step/sec: 321.986
INFO:tensorflow:loss = 0.0990827, step = 33401 (0.311 sec)
INFO:tensorflow:global_step/sec: 316.78
INFO:tensorflow:loss = 0.0855947, step = 33501 (0.316 sec)
INFO:tensorflow:global_step/sec: 330.202
INFO:tensorflow:loss = 0.0820529, step = 33601 (0.303 sec)
INFO:tensorflow:global_step/sec: 326.641
INFO:tensorflow:loss = 0.079173, step = 33701 (0.306 sec)
INFO:tensorflow:global_step/sec: 330.842
INFO:tensorflow:loss = 0.0771375, step = 33801 (0.302 sec)
INFO:tensorflow:global_step/sec: 331.048
INFO:tensorflow:loss = 0.070084, step = 33901 (0.302 sec)
INFO:tensorflow:global_step/sec: 329.054
INFO:tensorflow:loss = 0.0755171, step = 34001 (0.304 sec)
INFO:tensorflow:global_step/sec: 336.2
INFO:tensorflow:loss = 0.0829357, step = 34101 (0.297 sec)
INFO:tensorflow:global_step/sec: 327.241
INFO:tensorflow:loss = 0.0791803, step = 34201 (0.306 sec)
INFO:

...

INFO:tensorflow:global_step/sec: 320.617
INFO:tensorflow:loss = 0.00542105, step = 1281301 (0.314 sec)
INFO:tensorflow:global_step/sec: 325.024
INFO:tensorflow:loss = 0.00458171, step = 1281401 (0.305 sec)
INFO:tensorflow:global_step/sec: 325.833
INFO:tensorflow:loss = 0.00506999, step = 1281501 (0.307 sec)
INFO:tensorflow:global_step/sec: 338.137
INFO:tensorflow:loss = 0.00530838, step = 1281601 (0.296 sec)
INFO:tensorflow:global_step/sec: 332.852
INFO:tensorflow:loss = 0.00426717, step = 1281701 (0.300 sec)
INFO:tensorflow:global_step/sec: 319.796
INFO:tensorflow:loss = 0.00580996, step = 1281801 (0.313 sec)
INFO:tensorflow:global_step/sec: 330.11
INFO:tensorflow:loss = 0.00418258, step = 1281901 (0.303 sec)
INFO:tensorflow:global_step/sec: 333.331
INFO:tensorflow:loss = 0.00365388, step = 1282001 (0.300 sec)
INFO:tensorflow:global_step/sec: 333.595
INFO:tensorflow:loss = 0.00522145, step = 1282101 (0.300 sec)
INFO:tensorflow:global_step/sec: 324.56
INFO:tensorflow:loss = 0.00730531,

INFO:tensorflow:global_step/sec: 331.462
INFO:tensorflow:loss = 0.00452431, step = 1289301 (0.299 sec)
INFO:tensorflow:global_step/sec: 333.285
INFO:tensorflow:loss = 0.00399066, step = 1289401 (0.300 sec)
INFO:tensorflow:global_step/sec: 318.53
INFO:tensorflow:loss = 0.00469858, step = 1289501 (0.314 sec)
INFO:tensorflow:global_step/sec: 329.202
INFO:tensorflow:loss = 0.00392041, step = 1289601 (0.304 sec)
INFO:tensorflow:global_step/sec: 325.418
INFO:tensorflow:loss = 0.00440199, step = 1289701 (0.307 sec)
INFO:tensorflow:global_step/sec: 320.741
INFO:tensorflow:loss = 0.00348275, step = 1289801 (0.312 sec)
INFO:tensorflow:global_step/sec: 320.991
INFO:tensorflow:loss = 0.00428162, step = 1289901 (0.312 sec)
INFO:tensorflow:global_step/sec: 334.331
INFO:tensorflow:loss = 0.00615732, step = 1290001 (0.299 sec)
INFO:tensorflow:global_step/sec: 326.37
INFO:tensorflow:loss = 0.00449022, step = 1290101 (0.306 sec)
INFO:tensorflow:global_step/sec: 322.912
INFO:tensorflow:loss = 0.00534246,

INFO:tensorflow:global_step/sec: 320.675
INFO:tensorflow:loss = 0.00756306, step = 1297301 (0.312 sec)
INFO:tensorflow:global_step/sec: 334.138
INFO:tensorflow:loss = 0.00461957, step = 1297401 (0.299 sec)
INFO:tensorflow:global_step/sec: 331.885
INFO:tensorflow:loss = 0.0058389, step = 1297501 (0.301 sec)
INFO:tensorflow:global_step/sec: 333.162
INFO:tensorflow:loss = 0.0047725, step = 1297601 (0.300 sec)
INFO:tensorflow:global_step/sec: 332.578
INFO:tensorflow:loss = 0.00544624, step = 1297701 (0.301 sec)
INFO:tensorflow:global_step/sec: 328.464
INFO:tensorflow:loss = 0.00964928, step = 1297801 (0.304 sec)
INFO:tensorflow:global_step/sec: 315.698
INFO:tensorflow:loss = 0.00640915, step = 1297901 (0.317 sec)
INFO:tensorflow:global_step/sec: 320.772
INFO:tensorflow:loss = 0.00580241, step = 1298001 (0.312 sec)
INFO:tensorflow:global_step/sec: 332.616
INFO:tensorflow:loss = 0.00584126, step = 1298101 (0.301 sec)
INFO:tensorflow:global_step/sec: 326.22
INFO:tensorflow:loss = 0.00787083, 

INFO:tensorflow:global_step/sec: 332.307
INFO:tensorflow:loss = 0.0044968, step = 1305301 (0.301 sec)
INFO:tensorflow:global_step/sec: 328.605
INFO:tensorflow:loss = 0.00400101, step = 1305401 (0.304 sec)
INFO:tensorflow:global_step/sec: 332.786
INFO:tensorflow:loss = 0.00759521, step = 1305501 (0.300 sec)
INFO:tensorflow:global_step/sec: 322.139
INFO:tensorflow:loss = 0.00466169, step = 1305601 (0.311 sec)
INFO:tensorflow:global_step/sec: 332.436
INFO:tensorflow:loss = 0.00438102, step = 1305701 (0.301 sec)
INFO:tensorflow:global_step/sec: 328.685
INFO:tensorflow:loss = 0.00477001, step = 1305801 (0.304 sec)
INFO:tensorflow:global_step/sec: 327.223
INFO:tensorflow:loss = 0.00446003, step = 1305901 (0.306 sec)
INFO:tensorflow:global_step/sec: 335.822
INFO:tensorflow:loss = 0.00486746, step = 1306001 (0.298 sec)
INFO:tensorflow:global_step/sec: 334.462
INFO:tensorflow:loss = 0.00392032, step = 1306101 (0.299 sec)
INFO:tensorflow:global_step/sec: 334.558
INFO:tensorflow:loss = 0.00753988

INFO:tensorflow:global_step/sec: 336.595
INFO:tensorflow:loss = 0.00535124, step = 1313301 (0.297 sec)
INFO:tensorflow:global_step/sec: 338.373
INFO:tensorflow:loss = 0.00416882, step = 1313401 (0.296 sec)
INFO:tensorflow:global_step/sec: 335.736
INFO:tensorflow:loss = 0.00432463, step = 1313501 (0.298 sec)
INFO:tensorflow:global_step/sec: 334.779
INFO:tensorflow:loss = 0.00370984, step = 1313601 (0.299 sec)
INFO:tensorflow:global_step/sec: 322.891
INFO:tensorflow:loss = 0.005698, step = 1313701 (0.310 sec)
INFO:tensorflow:global_step/sec: 337.885
INFO:tensorflow:loss = 0.00380198, step = 1313801 (0.296 sec)
INFO:tensorflow:global_step/sec: 333.763
INFO:tensorflow:loss = 0.00339648, step = 1313901 (0.300 sec)
INFO:tensorflow:global_step/sec: 334.14
INFO:tensorflow:loss = 0.00541973, step = 1314001 (0.299 sec)
INFO:tensorflow:global_step/sec: 329.705
INFO:tensorflow:loss = 0.00414318, step = 1314101 (0.303 sec)
INFO:tensorflow:global_step/sec: 337.807
INFO:tensorflow:loss = 0.00306768, 

In [5]:
test = pd.read_csv("/dw-data/make-ipinyou-data/3358/test.log.txt",delimiter='\t',na_values=["null","NaN"],dtype="str")
test = test.drop(["bidid","urlid","ipinyouid","advertiser","logtype","url","bidprice","payprice","timestamp","keypage","nclick","nconversation"],axis=1)
test = test.dropna()
test["slotprice"] = test["slotprice"].astype(int)
test["click"] = test["click"].astype(int)
test_labels=test["click"]
test.head()

,click,weekday,hour,useragent,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,usertag
0,0,4,00,windows_chrome,60.187.41.*,94,100,1,trqRTuToMTNUjM9r5rMi,mm_17967481_3274913_10742094,950,90,0,1,0,cc9b344e950b4f8c2b96537174a343b7,10063
1,0,4,00,windows_chrome,111.179.205.*,183,193,1,trqRTummPvas1m58uG,mm_10058411_2865562_9929053,300,250,2,1,0,911b2d84826786018761e8c0b0a3a60c,"13042,10031,10063,10006,10075,10126,10116,10110"
2,0,4,00,windows_ie,110.73.196.*,238,248,1,DFpETuxoGQdcFNKbuKz,mm_10024662_3445902_11178345,336,280,2,1,0,dc0998c10f8f0b623b5d949e8272e4c7,"10006,10110"
3,0,4,00,windows_chrome,218.63.16.*,308,313,2,trqRTuMvjTN7X9KbuKz,3195670606,728,90,0,0,59,80446133fda50388678789e3ebd9fdb1,"13800,10133"
4,0,4,00,windows_ie,27.151.54.*,124,125,1,DFpETuxoGQdcFNKbuKz,mm_10024662_3445902_11178345,336,280,2,1,0,dc0998c10f8f0b623b5d949e8272e4c7,"10006,10110"


In [8]:
tf_test = tf.estimator.inputs.pandas_input_fn(
      x=test,
      y=test_labels,
      batch_size=128,
      num_epochs=1,
      shuffle=False,
      num_threads=1)
m.evaluate(tf_test)

INFO:tensorflow:Starting evaluation at 2017-09-15-09:12:05
INFO:tensorflow:Restoring parameters from ./linear_model/model.ckpt-1319249
INFO:tensorflow:Finished evaluation at 2017-09-15-09:12:13
INFO:tensorflow:Saving dict for global step 1319249: accuracy = 0.999105, accuracy_baseline = 0.999105, auc = 0.666826, auc_precision_recall = 0.00590115, average_loss = 0.00939127, global_step = 1319249, label/mean = 0.000895094, loss = 1.20154, prediction/mean = 0.00483734


{'accuracy': 0.99910492,
 'accuracy_baseline': 0.99910492,
 'auc': 0.66682613,
 'auc_precision_recall': 0.0059011541,
 'average_loss': 0.0093912659,
 'global_step': 1319249,
 'label/mean': 0.00089509413,
 'loss': 1.2015418,
 'prediction/mean': 0.0048373351}

In [9]:
tf_test = tf.estimator.inputs.pandas_input_fn(
      x=train,
      y=train_labels,
      batch_size=256,
      num_epochs=1,
      shuffle=False,
      num_threads=1)
m.evaluate(tf_test)

INFO:tensorflow:Starting evaluation at 2017-09-15-09:23:28
INFO:tensorflow:Restoring parameters from ./linear_model/model.ckpt-1319249
INFO:tensorflow:Finished evaluation at 2017-09-15-09:23:55
INFO:tensorflow:Saving dict for global step 1319249: accuracy = 0.999251, accuracy_baseline = 0.999251, auc = 0.613403, auc_precision_recall = 0.00403603, average_loss = 0.0089055, global_step = 1319249, label/mean = 0.000748911, loss = 2.27951, prediction/mean = 0.00505615


{'accuracy': 0.99925107,
 'accuracy_baseline': 0.99925107,
 'auc': 0.61340284,
 'auc_precision_recall': 0.004036027,
 'average_loss': 0.0089055039,
 'global_step': 1319249,
 'label/mean': 0.00074891094,
 'loss': 2.2795067,
 'prediction/mean': 0.0050561507}